In [69]:
from sqlalchemy import create_engine
from numpy import arange
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from neupy import algorithms

import warnings
warnings.filterwarnings('ignore')

import sys
sys.executable

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [70]:
localhost = {'user': 'postgres', 'password': 'postgres', 'host': 'localhost', 'port': 5432, 'db': 'fiadb'}
params = 'postgresql://{0}:{1}@{2}:{3}/{4}'
engine = create_engine(params.format(localhost['user'], localhost['password'], localhost['host'], localhost['port'], localhost['db']))
pergrid_base = """select distinct * from predictor.pergrid_base"""
pergrid_base_df = pd.read_sql(pergrid_base, engine)

In [71]:
pergrid_base_df.columns

Index(['grid_id', 'aet', 'ai', 'art', 'ewd', 'fa', 'map', 'mat', 'mpdq',
       'mtcq', 'pet', 'psn', 'ra', 'rmap', 'rmat', 'tsn', 'mfdf', 'alt', 'shg',
       'mtwq', 'wkb_geometry', 'tsr', 'wa', 'ha', 'wkt', 'lat', 'lon'],
      dtype='object')

In [72]:
y = pergrid_base_df['tsr']
grid_id = pergrid_base_df['grid_id']

In [73]:
var= ['grid_id', 'tsr', 'lat', 'lon', 'aet', 'ai', 'art', 'ewd', 'fa', 'map', 'mat', 'mpdq',
       'mtcq', 'pet', 'psn', 'ra', 'rmap', 'rmat', 'tsn', 'mfdf', 'alt','shg', 'mtwq', 'wa']
pred_var= ['aet', 'ai', 'art', 'ewd', 'fa', 'map', 'mat', 'mpdq',
       'mtcq', 'pet', 'psn', 'ra', 'rmap', 'rmat', 'tsn', 'mfdf', 'alt','shg', 'mtwq', 'wa']

In [74]:
pergrid_base_df = pergrid_base_df[var]

In [75]:
# Converting lat/long to cartesian
import numpy as np

def get_cartesian(lat=None,lon=None):
    lat, lon = np.deg2rad(lat), np.deg2rad(lon)
    R = 6371 # radius of the earth
    x = R * np.cos(lat) * np.cos(lon)
    y = R * np.cos(lat) * np.sin(lon)
    return x,y

In [76]:
pergrid_base_df['x'] = 0.0
pergrid_base_df['y'] = 0.0

In [77]:
for i in pergrid_base_df.index:
    lat = pergrid_base_df.at[i, 'lat']
    lon = pergrid_base_df.at[i, 'lon']
    x, y = get_cartesian(lat,lon)
    
    pergrid_base_df.at[i, 'x'] = x
    pergrid_base_df.at[i, 'y'] = y

In [78]:
pergrid_base_df.head()

,grid_id,tsr,lat,lon,aet,ai,art,ewd,fa,map,...,rmap,rmat,tsn,mfdf,alt,shg,mtwq,wa,x,y
0,110,4.0,25.932871,-97.631112,6000.0,0.3675,21.191667,-2.523666e+06,7.7058,681.0,...,2.833333,0.604166,444.860212,2.27,11.597701,None,28.792500,4.1166,-760.844723,-5678.742385
1,111,2.0,25.935809,-97.433331,65535.0,0.3829,20.233612,-2.537902e+06,14.1498,699.0,...,2.000000,0.620833,444.259982,2.10,5.982890,5,28.717458,11.8152,-741.219177,-5681.193259
2,195,5.0,24.833530,-80.942689,65535.0,NaN,0.164286,-5.247088e+02,0.0711,1148.5,...,1.833333,0.241667,3.698817,NaN,0.011696,None,28.626191,302.9049,910.197970,-5709.793273
3,337,1.0,26.099436,-98.427128,5135.0,0.2898,23.950167,-2.941595e+06,3.0042,565.0,...,2.333333,0.391667,492.281141,2.12,61.886973,None,29.458834,1.2528,-838.473507,-5659.588039
4,338,1.0,26.103897,-98.229000,5645.0,0.3017,23.526261,-2.959517e+06,10.0350,580.0,...,3.416667,0.620833,489.587254,2.12,32.270955,5,29.300416,4.2480,-818.866554,-5662.237626


In [84]:
var_new = ['grid_id', 'tsr', 'lat', 'lon', 'x', 'y', 'aet', 'ai', 'art', 'ewd', 'fa', 'map',
       'mat', 'mpdq', 'mtcq', 'pet', 'psn', 'ra', 'rmap', 'rmat', 'tsn',
       'mfdf', 'alt', 'shg', 'mtwq', 'wa']

In [80]:
from sklearn.preprocessing import StandardScaler

imputer = KNNImputer(n_neighbors=5)
pergrid_base_df_filled = imputer.fit_transform(pergrid_base_df)
pergrid_base_df = pd.DataFrame(pergrid_base_df_filled)

In [81]:
pergrid_base_df.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,110.0,4.0,25.932871,-97.631112,6000.0,0.36750,21.191667,-2.523666e+06,7.7058,681.0,...,2.833333,0.604166,444.860212,2.270,11.597701,4.6,28.792500,4.1166,-760.844723,-5678.742385
1,111.0,2.0,25.935809,-97.433331,65535.0,0.38290,20.233612,-2.537902e+06,14.1498,699.0,...,2.000000,0.620833,444.259982,2.100,5.982890,5.0,28.717458,11.8152,-741.219177,-5681.193259
2,195.0,5.0,24.833530,-80.942689,65535.0,1.30362,0.164286,-5.247088e+02,0.0711,1148.5,...,1.833333,0.241667,3.698817,3.892,0.011696,2.0,28.626191,302.9049,910.197970,-5709.793273
3,337.0,1.0,26.099436,-98.427128,5135.0,0.28980,23.950167,-2.941595e+06,3.0042,565.0,...,2.333333,0.391667,492.281141,2.120,61.886973,6.2,29.458834,1.2528,-838.473507,-5659.588039
4,338.0,1.0,26.103897,-98.229000,5645.0,0.30170,23.526261,-2.959517e+06,10.0350,580.0,...,3.416667,0.620833,489.587254,2.120,32.270955,5.0,29.300416,4.2480,-818.866554,-5662.237626


In [82]:
# standarize predictors
ss = MinMaxScaler()
pergrid_base_df_std = ss.fit_transform(pergrid_base_df)

pergrid_base_df_std=pd.DataFrame(data=pergrid_base_df_std[0:,0:],
                index=pergrid_base_df.index,
                columns=var_new)

In [85]:
pergrid_base_df_std.head()

,grid_id,tsr,lat,lon,x,y,aet,ai,art,ewd,...,psn,ra,rmap,rmat,tsn,mfdf,alt,shg,mtwq,wa
0,0.000000,0.050847,0.044925,0.469260,0.086844,0.085047,0.439220,0.452778,0.019513,0.190149,...,0.015733,0.033269,0.332249,0.015234,0.020341,0.600000,0.813142,0.010431,0.440653,0.013940
1,0.000031,0.016949,0.045045,0.472682,1.000000,0.088796,0.419208,0.449691,0.035830,0.195778,...,0.010816,0.034227,0.331797,0.009068,0.018803,0.666667,0.810267,0.029940,0.444996,0.012839
2,0.002598,0.067797,0.000000,0.757995,1.000000,0.312908,0.000000,0.999903,0.000180,0.336357,...,0.009833,0.012446,0.000000,0.074066,0.017168,0.166667,0.806770,0.767558,0.810472,0.000000
3,0.006939,0.000000,0.051732,0.455488,0.073577,0.066134,0.496839,0.362153,0.007607,0.153870,...,0.012783,0.021063,0.367963,0.009793,0.034114,0.866667,0.838671,0.003175,0.423473,0.022538
4,0.006970,0.000000,0.051914,0.458916,0.081399,0.069031,0.487985,0.358267,0.025411,0.158561,...,0.019174,0.034227,0.365934,0.009793,0.026003,0.666667,0.832601,0.010764,0.427812,0.021349


In [86]:
pergrid_base_df_test = pergrid_base_df_std.sample(frac=0.2, replace=False, random_state=1)
pergrid_base_df_train = pergrid_base_df_std.drop(pergrid_base_df_test.index)

In [87]:
pergrid_base_df_test.head()

,grid_id,tsr,lat,lon,x,y,aet,ai,art,ewd,...,psn,ra,rmap,rmat,tsn,mfdf,alt,shg,mtwq,wa
2158,0.259927,0.016949,0.296040,0.304620,0.008497,0.020933,0.620883,0.249902,0.000000,0.056919,...,0.029007,0.065342,0.445003,0.481683,0.390418,1.000000,0.719971,0.000000,0.256805,0.247546
2123,0.256137,0.372881,0.286553,0.699062,0.084314,0.187133,0.487311,0.702169,0.242673,0.365911,...,0.020157,0.023935,0.374497,0.199492,0.044924,0.333333,0.727239,0.006554,0.726977,0.145500
7905,0.537401,0.288136,0.601076,0.561508,0.089973,0.180074,0.702571,0.724429,0.293735,0.292103,...,0.011308,0.012207,0.647023,0.451941,0.082569,1.000000,0.626700,0.003795,0.565413,0.359463
10586,0.665454,0.305085,0.714132,0.695485,0.088424,0.179953,0.634699,0.844805,0.179996,0.234558,...,0.026549,0.008856,0.597047,0.573087,0.098707,0.000000,0.501974,0.003013,0.707968,0.457686
6617,0.479228,0.203390,0.531389,0.341469,0.037302,0.072585,0.558892,0.688113,0.504308,0.101225,...,0.184366,0.601963,0.438935,0.814291,0.693589,1.000000,0.331920,0.000032,0.320507,0.381620


In [91]:
X_test.head()

,aet,ai,art,ewd,fa,map,mat,mpdq,mtcq,pet,psn,ra,rmap,rmat,tsn,mfdf,alt,shg,mtwq,wa
2158,0.620883,0.249902,0.000000,0.056919,0.623931,0.035595,0.604319,0.713726,0.826143,0.104654,0.029007,0.065342,0.445003,0.481683,0.390418,1.000000,0.719971,0.000000,0.256805,0.247546
2123,0.487311,0.702169,0.242673,0.365911,0.668120,0.464961,0.678035,0.353022,0.246749,0.017207,0.020157,0.023935,0.374497,0.199492,0.044924,0.333333,0.727239,0.006554,0.726977,0.145500
7905,0.702571,0.724429,0.293735,0.292103,0.510524,0.289210,0.368770,0.241466,0.383196,0.016925,0.011308,0.012207,0.647023,0.451941,0.082569,1.000000,0.626700,0.003795,0.565413,0.359463
10586,0.634699,0.844805,0.179996,0.234558,0.377941,0.293660,0.300130,0.125283,0.292207,0.012976,0.026549,0.008856,0.597047,0.573087,0.098707,0.000000,0.501974,0.003013,0.707968,0.457686
6617,0.558892,0.688113,0.504308,0.101225,0.286555,0.094920,0.336528,0.338549,0.568664,0.498731,0.184366,0.601963,0.438935,0.814291,0.693589,1.000000,0.331920,0.000032,0.320507,0.381620


In [89]:
X_train=pergrid_base_df_train[pred_var]
Y_train=pergrid_base_df_train['tsr']

X_test=pergrid_base_df_test[pred_var]
Y_test=pergrid_base_df_test['tsr']

In [33]:
std_list = arange(0.05, 1, 0.005)
score_list = {}
for std in std_list:
    nw = algorithms.GRNN(std=std, verbose=False)
    nw.train(X_train, Y_train)
    Y_test_pred = nw.predict(X_test).flatten()
    mae = mean_absolute_error(Y_test_pred, Y_test)
    score_list.update({std:mae})

In [34]:
sorted_score_list = {k: v for k, v in sorted(score_list.items(), key=lambda item: item[1])}

In [36]:
nw = algorithms.GRNN(std=0.109, verbose=False) # 0.108 3.0961740739964703

In [45]:
nw.train(X_train, Y_train)

### Kriging

In [1]:
import numpy as np
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt

In [47]:
train_residual = nw.predict(X_train).flatten() - Y_train
train_df = pergrid_base_df_train[['grid_id', 'tsr', 'lat', 'lon', 'x', 'y']]
train_df['residual'] = train_residual.tolist()

In [92]:
pergrid_base_df_train.head()

,grid_id,tsr,lat,lon,x,y,aet,ai,art,ewd,...,psn,ra,rmap,rmat,tsn,mfdf,alt,shg,mtwq,wa
0,0.000000,0.050847,0.044925,0.469260,0.086844,0.085047,0.439220,0.452778,0.019513,0.190149,...,0.015733,0.033269,0.332249,0.015234,0.020341,0.600000,0.813142,0.010431,0.440653,0.013940
1,0.000031,0.016949,0.045045,0.472682,1.000000,0.088796,0.419208,0.449691,0.035830,0.195778,...,0.010816,0.034227,0.331797,0.009068,0.018803,0.666667,0.810267,0.029940,0.444996,0.012839
2,0.002598,0.067797,0.000000,0.757995,1.000000,0.312908,0.000000,0.999903,0.000180,0.336357,...,0.009833,0.012446,0.000000,0.074066,0.017168,0.166667,0.806770,0.767558,0.810472,0.000000
3,0.006939,0.000000,0.051732,0.455488,0.073577,0.066134,0.496839,0.362153,0.007607,0.153870,...,0.012783,0.021063,0.367963,0.009793,0.034114,0.866667,0.838671,0.003175,0.423473,0.022538
7,0.007092,0.016949,0.052488,0.472631,0.091400,0.091960,0.434137,0.436386,0.035342,0.196716,...,0.009341,0.027764,0.347309,0.016685,0.018599,0.333333,0.803086,0.017793,0.445185,0.016813


In [51]:
test_residual = nw.predict(X_test).flatten() - Y_test
test_df = pergrid_base_df_test[['grid_id', 'tsr', 'lat', 'lon', 'x', 'y']]
test_df['residual'] = test_residual.tolist()

In [61]:
test_df['predicted_tsr'] = nw.predict(pergrid_base_df_test[pred_var])

In [62]:
test_df.head(2)

,grid_id,tsr,lat,lon,x,y,residual,residual_pred,predicted_tsr
2158,0.259927,0.016949,0.296040,0.304620,0.008497,0.020933,-0.009382,0.000023,0.007568
2123,0.256137,0.372881,0.286553,0.699062,0.084314,0.187133,0.065829,-0.000203,0.438710


In [55]:
OK = OrdinaryKriging(
    x=train_df['x'], #'lon'
    y=train_df['y'], # 'lat'
    z=train_df['residual'],
    coordinates_type='euclidean', #'geographic'
    variogram_model="linear",
    verbose=False,
    enable_statistics=False
)

In [57]:
predict_value_list, ss = OK.execute("points", test_df['x'], test_df['y'])

In [58]:
test_df['residual_pred'] = predict_value_list

In [63]:
test_df['predicted_tsr_kriging'] = test_df['predicted_tsr'] + test_df['residual_pred']

In [64]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [66]:
print('test MAE', mean_absolute_error(test_df['predicted_tsr'], test_df['tsr']))
print('test r2', r2_score(test_df['tsr'], test_df['predicted_tsr']))

test MAE 0.05146710863936504
test r2 0.889084717775072


In [65]:
print('test MAE', mean_absolute_error(test_df['predicted_tsr_kriging'], test_df['tsr']))
print('test r2', r2_score(test_df['tsr'], test_df['predicted_tsr_kriging']))

test MAE 0.0517133234484143
test r2 0.8880365930615092


In [67]:
test_df.head()

,grid_id,tsr,lat,lon,x,y,residual,residual_pred,predicted_tsr,predicted_tsr_kriging
2158,0.259927,0.016949,0.296040,0.304620,0.008497,0.020933,-0.009382,0.000023,0.007568,0.007591
2123,0.256137,0.372881,0.286553,0.699062,0.084314,0.187133,0.065829,-0.000203,0.438710,0.438508
7905,0.537401,0.288136,0.601076,0.561508,0.089973,0.180074,0.063407,-0.000216,0.351542,0.351326
10586,0.665454,0.305085,0.714132,0.695485,0.088424,0.179953,0.055539,-0.000215,0.360624,0.360409
6617,0.479228,0.203390,0.531389,0.341469,0.037302,0.072585,-0.030915,-0.000021,0.172475,0.172453


In [384]:
grnn_y_test = pd.DataFrame(
    {'tsr': Y_test,
     'tsr_predicted': Y_test_pred})
grnn_y_test.to_sql(name='grnn_y_test', con=engine, schema='predictor', if_exists='replace', index=False)

### Ingest TSR predictions to PG

In [385]:
y_pred = nw.predict(xstd).flatten()

In [386]:
pergrid_all_predicted = pd.DataFrame(
    {'grid_id': grid_id,
     'tsr': y,
     'tsr_predicted': y_pred})

In [387]:
pergrid_all_predicted.head()

,grid_id,tsr,tsr_predicted
0,110,4.0,3.276433
1,111,2.0,2.000123
2,195,5.0,4.994345
3,337,1.0,2.803938
4,338,1.0,2.795763


In [388]:
pergrid_all_predicted.to_sql(name='grnn', con=engine, schema='predictor', if_exists='replace', index=False)

In [389]:
update_geom = """
alter table predictor.grnn add column if not exists wkb_geometry geometry(Polygon,4269);
update predictor.grnn A SET wkb_geometry = B.wkb_geometry
FROM predictor.pergrid_base B
WHERE A.grid_id = B.grid_id
"""

In [390]:
connection = engine.connect()
connection.execute(update_geom)

In [391]:
update_residual = """
alter table predictor.grnn add column residual double precision;
update predictor.grnn set residual = (tsr_predicted-tsr);
"""

In [392]:
connection = engine.connect()
connection.execute(update_residual)